In [1]:
# Here we take care of paths.
# Make sure root project directory is named 'VESUVIUS_Challenge' for this to work

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in the root folder of the project
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [8]:
import torch
import monai
#from monai.visualize import matshow3d
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, List
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Dataset import Vesuvius_Tile_Datamodule
from lit_models.Vesuvius_Lit_Model import Lit_Model
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn as nn
from Models.PVT2 import PyramidVisionTransformerV2, Up, OutConv
import torch.nn as nn
from functools import partial
import torchvision
import torch.nn.functional as F
from Models.Swin import SwinTransformer, SwinTransformerBlockV2, PatchMergingV2
from lit_models.Loss_functions import ComboBCEDiceLoss, TverskyLoss
from lit_models.scratch_models import FPNDecoder, Feature2Pyramid
from segmentation_models_pytorch.base import SegmentationHead


In [9]:
PATCH_SIZE = 256
Z_DIM = 8
COMPETITION_DATA_DIR_str =  "kaggle/input/vesuvius-challenge-ink-detection/"


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

# change to the line below if not using Apple's M1 or chips
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
class PVT_w_FPN(nn.Module):
    def __init__(self, in_channels,  embed_dims=[  64, 128, 256, 512], n_classes=1, ):
        super().__init__()
        
        self.embed_dims = embed_dims
        
       
        self.pvt = PyramidVisionTransformerV2(img_size = PATCH_SIZE,
                                  patch_size = 4,
                                  in_chans = Z_DIM,
                                  num_classes = 1,
                                  embed_dims = embed_dims,
                                num_heads=[1, 2, 4, 8],
                                  mlp_ratios=[4, 4, 4, 4],
                                  qkv_bias=True,
                                  qk_scale=None,
                                  drop_rate=0.,
                                attn_drop_rate=0.,
                                  drop_path_rate=0.1,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-3),
                                #norm_layer=nn.LayerNorm,          
                                  depths=[2, 2, 2,2],
                                  sr_ratios=[1, 1, 1, 1]
                                 ).to(DEVICE) 
        
        self.FPN = FPNDecoder(
                            in_channels = Z_DIM,
                            encoder_channels = embed_dims ,
                            encoder_depth=5,
                            pyramid_channels=256,
                            segmentation_channels=128,
                            dropout=0.2,
                            merge_policy="cat",).to(DEVICE) 
        
       

    def forward(self, x):
        #x = x.unsqueeze(1)
        #x = self.pre_model3d(x)
        #x = x.squeeze(1)
        
        pvt_outs = self.pvt(x)
        
        logits = self.FPN(*pvt_outs)
        
       
       
            
        
        return logits












In [19]:
dummy = torch.randn(5,8,256,256).to(DEVICE) 
model = PVT_w_FPN(in_channels =8 ,  embed_dims=[ 64, 128, 256, 512])
out = model(dummy)
print(out.shape)

torch.Size([5, 1, 256, 256])


# BACKBONE

In [16]:
dummy = torch.randn(5,8,256,256).to(DEVICE) 
pvt_outs = model.pvt(dummy)
print('pvt outputs')
for t in pvt_outs:
    print(t.shape)

pvt outputs
torch.Size([5, 8, 256, 256])
torch.Size([5, 64, 64, 64])
torch.Size([5, 128, 32, 32])
torch.Size([5, 256, 16, 16])
torch.Size([5, 512, 8, 8])


# NECK FPN

In [17]:
fpn_outs = model.FPN(*pvt_outs)
print(fpn_outs.shape)

torch.Size([5, 1, 256, 256])
